In [1]:
import pandas as pd
from datetime import datetime
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from postgresql.connection import create_connection, close_connection
from config import settings

conn = create_connection(
            settings.database.database_name,
            settings.database.username,
            settings.database.password,
            settings.database.host,
            settings.database.port,
        )


events=pd.read_sql('Select * from events',conn)
nearby_cities=pd.read_sql('Select * from nearby_cities',conn)

Connection to PostgreSQL DB successful


### Q6. Biggest earthquake in 2017

In [4]:
events_with_max_magnitude=pd.read_sql('select "property.mag", "property.place", "geometry.coordinates",created_at from events where "property.mag" = (select max("property.mag") from events);',conn)
events_with_max_magnitude

,property.mag,property.place,geometry.coordinates,created_at
0,6.0,"195 km SW of Nadi, Fiji","[176, -19, 10]",2017-01-01 00:04:06+00:00
1,6.0,"225 km SW of Nadi, Fiji","[176, -19, 12]",2017-01-01 00:04:06+00:00
2,6.0,south of the Fiji Islands,"[179, -23, 552]",2017-01-01 00:04:06+00:00


In [5]:
def get_magnitude_bucket(x: float):
    """
    Given a magnitude, this method decides magnitude bucket
    following the rule
    """
    if x < 0: 
        return '0-1'
    elif x > 6: 
        return '>6'
    else: 
        return str(math.floor(x))+'-'+str(math.floor(x)+1)

#### Identify magnitude_bucket for each earthquake event

In [6]:
events['hour'] = events['created_at'].apply(lambda x: x.hour)
events['magnitude_bucket'] = events['property.mag'].apply (lambda x: get_magnitude_bucket(x))
events.head()

,id,type,property.mag,property.place,property.detail,geometry.type,geometry.coordinates,generated_at,created_at,updated_at,hour,magnitude_bucket
0,us10007pke,Feature,5.0,"174 km SW of Nadi, Fiji",https://earthquake.usgs.gov/fdsnws/event/1/que...,Point,"[176, -19, 10]",1668014872000,2017-01-01 00:04:06+00:00,2017-02-08 21:15:08+00:00,0,5-6
1,ak0175clbmm,Feature,1.0,"15 km SSE of Susitna North, Alaska",https://earthquake.usgs.gov/fdsnws/event/1/que...,Point,"[-150, 62, 46]",1668014872000,2017-01-01 00:04:06+00:00,2017-02-08 21:15:08+00:00,0,1-2
2,us10007ue3,Feature,5.0,"220 km SW of Nadi, Fiji",https://earthquake.usgs.gov/fdsnws/event/1/que...,Point,"[176, -19, 10]",1668014872000,2017-01-01 00:04:06+00:00,2017-02-08 21:15:08+00:00,0,5-6
3,us10007pkb,Feature,5.0,"191 km SW of Nadi, Fiji",https://earthquake.usgs.gov/fdsnws/event/1/que...,Point,"[176, -19, 10]",1668014872000,2017-01-01 00:04:06+00:00,2017-02-08 21:15:08+00:00,0,5-6
4,us10007udt,Feature,4.0,Fiji region,https://earthquake.usgs.gov/fdsnws/event/1/que...,Point,"[-178, -18, 630]",1668014872000,2017-01-01 00:04:06+00:00,2017-02-08 21:15:08+00:00,0,4-5


#### The following cell depicts the earthquake frequency per magnitude bucket and per hour of the day

In [42]:
earthquake_freq_per_bucket_hr=events.groupby(['magnitude_bucket', 'hour']).size().to_frame('earthquake_freq').reset_index()
earthquake_freq_per_bucket_hr.head()

,magnitude_bucket,hour,earthquake_freq
0,0-1,0,561
1,1-2,0,797
2,2-3,0,137
3,3-4,0,29
4,4-5,0,129


### Q7.  The highest probable hour of the day for earthquakes bucketed by magnitudes of earthquake

In [68]:
indices_to_fetch = earthquake_freq_per_bucket_hr.groupby(['hour'])['earthquake_freq'].transform(max) == earthquake_freq_per_bucket_hr['earthquake_freq']
earthquake_freq_per_bucket_hr[indices_to_fetch]

,magnitude_bucket,hour,earthquake_freq
1,1-2,0,797
